In [1]:
from cbrain.imports import *
from cbrain.data_generator import *
from cbrain.cam_constants import *
from cbrain.losses import *
from cbrain.utils import limit_mem
from cbrain.layers import *
from cbrain.data_generator import DataGenerator
import tensorflow as tf
from tensorflow import math as tfm
#import tensorflow_probability as tfp
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin
#import cartopy.crs as ccrs
import matplotlib.ticker as mticker
#from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import pickle
import sklearn
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
from cbrain.imports import *
from cbrain.utils import *
from cbrain.normalization import *
import h5py
from sklearn.preprocessing import OneHotEncoder
from cbrain.climate_invariant import *
import yaml
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# Load coordinates (just pick any file from the climate model run)
coor = xr.open_dataset("/DFS-L/DATA/pritchard/ankitesg/data/CESM2_f19_v13_updated_NN_pelayout01_ens_07.cam.h1.2003-01-22-00000.nc",\
                    decode_times=False)
lat = coor.lat; lon = coor.lon; lev = coor.lev;
DATA_DIR = '/DFS-L/DATA/pritchard/ankitesg/datav2/'
hyam = coor.hyam
hybm = coor.hybm
scale_dict = load_pickle('/export/nfs0home/ankitesg/tom/CBRAIN-CAM/nn_config/scale_dicts/2020_10_16_scale_dict_RG.pkl')['scale_dict_RG']

In [3]:
class DataGeneratorClimInvRealGeo(DataGenerator):

    def __init__(self, data_fn, input_vars, output_vars,
             norm_fn=None, input_transform=None, output_transform=None,
             batch_size=1024, shuffle=True, xarray=False, var_cut_off=None, normalize_flag=True,
             rh_trans=True,t2tns_trans=True,
             lhflx_trans=True,
             scaling=True,interpolate=True,
             hyam=None,hybm=None,
             inp_subRH=None,inp_divRH=None,
             inp_subTNS=None,inp_divTNS=None,
             lev=None, interm_size=40,
             lower_lim=6,
             is_continous=True,Tnot=5,
                mode='train', exp=None):
        self.scaling = scaling
        self.interpolate = interpolate
        self.rh_trans = rh_trans
        self.t2tns_trans = t2tns_trans
        self.lhflx_trans = lhflx_trans
        self.inp_shape = 64
        self.exp = exp
        self.mode=mode
        super().__init__(data_fn, input_vars,output_vars,norm_fn,input_transform,output_transform,
                        batch_size,shuffle,xarray,var_cut_off,normalize_flag) ## call the base data generator
        self.inp_sub = self.input_transform.sub
        self.inp_div = self.input_transform.div


    def __getitem__(self, index):
        # Compute start and end indices for batch
        start_idx = index * self.batch_size
        end_idx = start_idx + self.batch_size

        # Grab batch from data
        batch = self.data_ds['vars'][start_idx:end_idx]

        # Split into inputs and outputs
        X = batch[:, self.input_idxs]
        Y = batch[:, self.output_idxs]
        # Normalize
        X_norm = self.input_transform.transform(X)
        Y = self.output_transform.transform(Y)
        return X_norm, Y

In [4]:
in_vars = ['QBP','TBP','CLDLIQBP','CLDICEBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['QBCTEND','TBCTEND','CLDLIQBCTEND', 'CLDICEBCTEND', 'NN2L_FLWDS', 'NN2L_PRECC', 
            'NN2L_PRECSC', 'NN2L_SOLL', 'NN2L_SOLLD', 'NN2L_SOLS', 'NN2L_SOLSD', 'NN2L_NETSW']

In [5]:
TRAINFILE = 'RG_SP_M4K_train_shuffle.nc'
NORMFILE = 'RG_SP_M4K_NORM_norm.nc'
VALIDFILE = 'RG_SP_M4K_valid.nc'

In [6]:
train_gen_bf = DataGeneratorClimInv(
    data_fn = f'{DATA_DIR}{TRAINFILE}',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = f'{DATA_DIR}{NORMFILE}',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    rh_trans = False,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False
)


In [7]:
valid_gen_bf = DataGeneratorClimInv(
    data_fn = f'{DATA_DIR}{VALIDFILE}',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = f'{DATA_DIR}{NORMFILE}',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    rh_trans = False,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False
)


### Network

In [111]:
path_HDF5 = '/DFS-L/DATA/pritchard/ankitesg/models/'

In [112]:
from keras import backend as K
def partial_relu(x):
    a = np.zeros((1,112))
    a[:,104:] = 1
    not_a = 1-a
    a = K.constant(a)
    not_a = K.constant(not_a)
    acti = not_a*x +  a*K.relu(x)
    return acti

In [113]:
inp = Input(shape=(108,))
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(112, activation=partial_relu)(densout)
model = tf.keras.models.Model(inp, dense_out)

In [114]:
model.compile(optimizer='adam', loss='mse')

In [115]:
model.summary()

Model: "functional_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 108)]             0         
_________________________________________________________________
dense_88 (Dense)             (None, 128)               13952     
_________________________________________________________________
leaky_re_lu_77 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_89 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_78 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_90 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_79 (LeakyReLU)   (None, 128)             

In [116]:
# model.compile(tf.keras.optimizers.Adam(), loss="mse")
path_HDF5 = '/DFS-L/DATA/pritchard/ankitesg/models/'
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint(path_HDF5+'BF_RG_PartialRelu.h5',save_best_only=True, monitor='val_loss', mode='min')

In [117]:
with tf.device('/gpu:0'):
    Nep = 5
    model.fit_generator(train_gen_bf, epochs=Nep, validation_data=valid_gen_bf,\
                  callbacks=[earlyStopping, mcp_save])

Epoch 1/5
16200/16200 [==============================] - 236s 15ms/step - batch: 8099.5000 - size: 1024.0000 - loss: 562.8188 - val_loss: 228.9155
Epoch 2/5
16200/16200 [==============================] - 218s 13ms/step - batch: 8099.5000 - size: 1024.0000 - loss: 216.2452 - val_loss: 210.5763
Epoch 3/5
16200/16200 [==============================] - 196s 12ms/step - batch: 8099.5000 - size: 1024.0000 - loss: 204.2939 - val_loss: 203.2969
Epoch 4/5
16200/16200 [==============================] - 208s 13ms/step - batch: 8099.5000 - size: 1024.0000 - loss: 197.7993 - val_loss: 200.3087
Epoch 5/5
16200/16200 [==============================] - 216s 13ms/step - batch: 8099.5000 - size: 1024.0000 - loss: 193.4987 - val_loss: 194.6090


### How the layers will look when loading for conversion using FKB

In [119]:
weights_file = h5py.File(path_HDF5+'BF_RG_PartialRelu.h5','r')
model_weights = weights_file['model_weights']  
model_config = weights_file.attrs['model_config'].decode('utf-8')

In [120]:
model_config

'{"class_name": "Functional", "config": {"name": "functional_23", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 108], "dtype": "float32", "sparse": false, "ragged": false, "name": "input_12"}, "name": "input_12", "inbound_nodes": []}, {"class_name": "Dense", "config": {"name": "dense_88", "trainable": true, "dtype": "float32", "units": 128, "activation": "linear", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null, "dtype": "float32"}}, "bias_initializer": {"class_name": "Zeros", "config": {"dtype": "float32"}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}, "name": "dense_88", "inbound_nodes": [[["input_12", 0, 0, {}]]]}, {"class_name": "LeakyReLU", "config": {"name": "leaky_re_lu_77", "trainable": true, "dtype": "float32", "alpha": 0.30000001192092896}, "name": "leaky_re_lu_77", "inbound_nodes": [[["dense_88", 0, 0, {}

In [121]:
model_config = model_config.replace('true','True')
model_config = model_config.replace('false','False')
model_config = model_config.replace('null','None')
model_config = eval(model_config)


In [122]:
model_config

{'class_name': 'Functional',
 'config': {'name': 'functional_23',
  'layers': [{'class_name': 'InputLayer',
    'config': {'batch_input_shape': [None, 108],
     'dtype': 'float32',
     'sparse': False,
     'ragged': False,
     'name': 'input_12'},
    'name': 'input_12',
    'inbound_nodes': []},
   {'class_name': 'Dense',
    'config': {'name': 'dense_88',
     'trainable': True,
     'dtype': 'float32',
     'units': 128,
     'activation': 'linear',
     'use_bias': True,
     'kernel_initializer': {'class_name': 'GlorotUniform',
      'config': {'seed': None, 'dtype': 'float32'}},
     'bias_initializer': {'class_name': 'Zeros',
      'config': {'dtype': 'float32'}},
     'kernel_regularizer': None,
     'bias_regularizer': None,
     'activity_regularizer': None,
     'kernel_constraint': None,
     'bias_constraint': None},
    'name': 'dense_88',
    'inbound_nodes': [[['input_12', 0, 0, {}]]]},
   {'class_name': 'LeakyReLU',
    'config': {'name': 'leaky_re_lu_77',
     'tr